In [ ]:
#| default_exp mocking.functions

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import inspect
from typing import Any

from pymoq.core import AnyCallable
from pymoq.argument_validators import ArgumentFunctionValidator
from pymoq.signature_validators import SignatureValidator, signature_validator_from_arguments
from pymoq.return_value_generators import ReturnValueGenerator

from fastcore.basics import patch_to

In [ ]:
#| hide
from fastcore.test import test_fail
from nbdev.showdoc import show_doc

# Mocking functions
> From function object to function mock

## Checking arguments against original signature

For a given argument list, we want to know whether the original function can be called with that set of parameters. This can be done in two steps:
1. Extract the signature
2. Try to bind the argument list against the signature. This throws an exception if the arguments can not be matched against the signature

### Binding free methods

In [ ]:
def f(a: int, b: str, c:str|None =None) -> None:
    pass

Successful bind:

In [ ]:
inspect.signature(f).bind(1, "1", "2")

<BoundArguments (a=1, b='1', c='2')>

Unsuccessful bind:

In [ ]:
try:
    inspect.signature(f).bind(1)
except Exception as e:
    print(e)

missing a required argument: 'b'


### Binding class methods

When binding class methods, the special case of the self parameter has to be considered.

In [ ]:
class A:
    def f(self, a:int):
        return a+1

In [ ]:
signature = inspect.signature(A.f)

try:
    signature.bind(1)
except Exception as e:
    print(e)    

missing a required argument: 'a'


We will mark a function as 'class'-function if it has a parameter called 'self'.

In [ ]:
#| export
def is_class_method(func: AnyCallable) -> bool:
    "Returns true if the given function has a parameter called 'self'"
    signature = inspect.signature(func)
    return 'self' in signature.parameters

In [ ]:
assert is_class_method(A.f)
assert not is_class_method(f)

In [ ]:
#| export
def add_self_parameter(args: tuple[Any]) -> list[Any]:
    return (None,) + args

### Function Mock

In [ ]:
#| export
class FunctionMock:
    "Mocks a function object based on its signature"
    def __init__(self, func: AnyCallable):
        self._func = func
        self._signature = inspect.signature(self._func)
        self._setups = []
        
        self._is_class_method = is_class_method(self._func)
        
    def arguments_valid(self, *args, **kwargs) -> None:
        "Given an arbitrary argument list (both positional and keyword arguments), returns True if the mocked function could be called with those arguments"
        self._signature.bind(*args, **kwargs)
        return True

In [ ]:
mock = FunctionMock(f)

Successful binds:

In [ ]:
assert mock.arguments_valid(1, "1", "2")
assert mock.arguments_valid(1, "1")
assert mock.arguments_valid(1, "1", c="2")
assert mock.arguments_valid(a=1, b="1", c="2")

Note that the types are not checked with this:

In [ ]:
assert mock.arguments_valid(1, 1)

Unsuccessful binds might be:

In [ ]:
def show_exception(func: AnyCallable):
    try:
        func()
    except Exception as e:
        print(e)
        return
    assert False, "Expected Exception to be thrown"

In [ ]:
show_exception(lambda: mock.arguments_valid(1))# too few arguments

missing a required argument: 'b'


In [ ]:
show_exception(lambda:mock.arguments_valid(1,2,3,4)) # too many arguments

too many positional arguments


In [ ]:
show_exception(lambda: mock.arguments_valid(1,2,d=3)) # unknown argument name

got an unexpected keyword argument 'd'


Special case class method. Note that when calling `arguments_valid` directly, we need to pass in a dummy `self` argument.

In [ ]:
mock = FunctionMock(A.f)

assert mock.arguments_valid(None, 1)
show_exception(lambda: mock.arguments_valid(None))
show_exception(lambda: mock.arguments_valid(None, 1,2))

missing a required argument: 'a'
too many positional arguments


## Setup

We want to be able to create call-setups on the function mock. A setup consists of a signature validation and a return value generator. When the mock is called with a list of arguments, we check this list against the signature validator. If the call matches, we call the return value generator to generate the return value.

In [ ]:
#| export
class Setup:
    "This class bundles a signature validator with a call-result-action"
    def __init__(self, signature_validator: SignatureValidator):
        self._signature_validator = signature_validator
        
    def is_valid(self, *args, **kwargs) -> bool:
        "Uses the underlying `SignatureValidator` to determine if the argument list is valid"
        return self._signature_validator.is_valid(*args, **kwargs)
        
    def returns(self, return_value_generator: ReturnValueGenerator) -> None:
        "Set the `ReturnValueGenerator` to be called when this setup is successfully called"
        self._return_value_generator = return_value_generator
        
    def get_return_value(self, *args, **kwargs):
        "Calls the underlying `ReturnValueGenerator` the get the return value for the exact argument list"
        return self._return_value_generator(*args, **kwargs)

In [ ]:
show_doc(Setup.returns)

---

### Setup.returns

>      Setup.returns (return_value_generator:pymoq.return_value_generators.Retur
>                     nValueGenerator)

Set the `ReturnValueGenerator` to be called when this setup is successfully called

In [ ]:
#| export
@patch_to(FunctionMock)
def setup(self, *args, **kwargs):
    # todo: actually implement this
    sig = signature_validator_from_arguments(*args, **kwargs)
    self._setups.append(Setup(sig))
    
    return self._setups[-1]

In [ ]:
mock = FunctionMock(f)

mock.setup(ArgumentFunctionValidator(lambda a: isinstance(a, int), name='a', position=0)).returns(lambda *args, **kwargs: 5)
assert mock._setups[0].get_return_value() == 5

In [ ]:
assert mock._setups[0].get_return_value(1)==5
assert mock._setups[0].get_return_value(1, a=1)==5

In [ ]:
assert mock._setups[0].is_valid(1)
assert not mock._setups[0].is_valid("1")

## Call validation

When called, a function mock should perform the following steps:
1. check if the argument list binds against the original functions signature
2. check if the signature validator matches for the stored Setups in order they were added
3. The first setup with a matching signature validator should be used for generating the return value

Edge cases:
- If no Setup matches, return None

### Dealing with default-values

The number of arguments passed to a function call might differ from call to call if default values are used:

In [ ]:
def f(a:int, b:str="str"):
    ...
    
mock = FunctionMock(f)
mock.setup(ArgumentFunctionValidator(lambda a: True, name='a', position=0), ArgumentFunctionValidator(lambda b: True, name='b', position=1)).returns(lambda *args,**kwargs: print(args, kwargs))

In [ ]:
mock._setups[0].get_return_value(1), mock._setups[0].get_return_value(1, b="b")

(1,) {}
(1,) {'b': 'b'}


(None, None)

Next, we'll match an argument list against the default values.

In [ ]:
#| export
@patch_to(FunctionMock)
def fill_up_arg_list(self, args: list[Any], kwargs: dict[str, Any]) -> dict[str, Any]:
    parameters = list(self._signature.parameters.items())
    names = list(map(lambda p: p[0], parameters))
    n_positional = len(args)
    for name,param in parameters[n_positional:]:
        if name in kwargs: continue
        kwargs[name] = param.default
        
    return kwargs

If a default-value is already present via name, the default value should not be used:

In [ ]:
def f(a: int, b: str, c:str|None ='default str') -> None:
    pass

mock = FunctionMock(f)

assert mock.fill_up_arg_list([1, 1.1], {'c': 'custom str'}) == {'c': 'custom str'}

If a default-value is already overriden by a positional argument, it should not be used:

In [ ]:
assert mock.fill_up_arg_list([1, 1.1, 'custom str'], {}) == {}

If neither is the case, the default-value should be used:

In [ ]:
assert mock.fill_up_arg_list([1, 1.1], {}) == {'c': 'default str'}

### Call method

In [ ]:
#| export
@patch_to(FunctionMock)
def __call__(self, *args, **kwargs):
    if self._is_class_method:
        args = add_self_parameter(args)
    
    self.arguments_valid(*args, **kwargs)
        
    kwargs = self.fill_up_arg_list(args, kwargs)
    
    for setup in reversed(self._setups):
        if setup.is_valid(*args, **kwargs):
            return_value = setup.get_return_value(*args, **kwargs)
            return return_value

NameError: name 'patch_to' is not defined

In [ ]:
def f(a: int, b: str, c:str|None =None) -> None:
    pass

mock = FunctionMock(f)

Without any setup a call either fails or returns None:

In [ ]:
test_fail(lambda: mock())
assert mock(1,"1") is None

(1, '1') {'c': None}


Now we'll add two setups. The first one is the more generic one that only checks if `a` is an `int` and `b` is a `string`. The second one checks if `a==2` and `b` is a string.

In [ ]:
# this will be prettier, I promise!
mock = FunctionMock(f)

# Generic type checker
mock.setup(
    ArgumentFunctionValidator(lambda a: isinstance(a, int), name='a', position=0),
    ArgumentFunctionValidator(lambda b: isinstance(b, str), name='b', position=1),
    ArgumentFunctionValidator(lambda c: c is None, name='c', position=2)).returns(lambda a,b,c: 5)

# Value checker
mock.setup(
    ArgumentFunctionValidator(lambda a: a==2, name='a', position=0),
    ArgumentFunctionValidator(lambda b: isinstance(b, str), name='b', position=1),
    ArgumentFunctionValidator(lambda c: c is None, name='c', position=2)).returns(lambda a,b,c: 6)

assert mock(2, 'anyString')==6
assert mock(1, 'anyString')==5
assert mock(1, 1) is None

(2, 'anyString') {'c': None}
(1, 'anyString') {'c': None}
(1, 1) {'c': None}


Lastly, the generic return value generator could be used to return a value based on the input value:

In [ ]:
mock = FunctionMock(f)

# Generic type checker
mock.setup(
    ArgumentFunctionValidator(lambda a: isinstance(a, int), name='a', position=0),
    ArgumentFunctionValidator(lambda b: isinstance(b, str), name='b', position=1),
    ArgumentFunctionValidator(lambda c: c is None, name='c', position=2)).returns(lambda a,b,c: a+1)

assert mock(1,'anyString')==2
assert mock(2,'anyString')==3

(1, 'anyString') {'c': None}
(2, 'anyString') {'c': None}


### On class methods

In [ ]:
class A:
    def f(self, a: int, b: str, c:str|None =None) -> None:
        pass

In [ ]:
mock = FunctionMock(A.f)

In [ ]:
test_fail(lambda: mock())
assert mock(1, "1", "c") is None

(None, 1, '1', 'c') {}


In [ ]:
# this will be prettier, I promise!
mock = FunctionMock(A.f)

# Generic type checker
mock.setup(
    ArgumentFunctionValidator(lambda s: True, name='self', position=0),
    ArgumentFunctionValidator(lambda a: isinstance(a, int), name='a', position=1),
    ArgumentFunctionValidator(lambda b: isinstance(b, str), name='b', position=2),
    ArgumentFunctionValidator(lambda c: c is None, name='c', position=3)).returns(lambda self,a,b,c: 5)

# Value checker
mock.setup(
    ArgumentFunctionValidator(lambda s: True, name='self', position=0),
    ArgumentFunctionValidator(lambda a: a==2, name='a', position=1),
    ArgumentFunctionValidator(lambda b: isinstance(b, str), name='b', position=2),
    ArgumentFunctionValidator(lambda c: c is None, name='c', position=3)).returns(lambda self,a,b,c: 6)

assert mock(2, 'anyString')==6
assert mock(1, 'anyString')==5
assert mock(1, 1) is None

(None, 2, 'anyString') {'c': None}
(None, 1, 'anyString') {'c': None}
(None, 1, 1) {'c': None}


# Build library

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()